In [2]:
!pip install tokenizers
import pandas as pd
import re
from tokenizers import Tokenizer, models, trainers, normalizers, pre_tokenizers, processors

# Step 1: Read the dataset from CSV
df = pd.read_csv(r'C:\Users\akash\Python_DL\project_data\Data.csv')
text_data = df["Message"].tolist()

# Step 2: Preprocess the text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    return text

preprocessed_text_data = [preprocess_text(text) for text in text_data]

# Step 3: Prepare the training corpus
training_corpus = preprocessed_text_data

# Step 4: Train the tokenizer
tokenizer = Tokenizer(models.WordPiece())

tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

tokenizer.train_from_iterator(training_corpus, trainer=trainer)

# Optionally, you can save the trained tokenizer for future use
tokenizer.save("custom_tokenizer.json")


In [8]:
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, processors, decoders
import re

# Load the trained tokenizer from the saved file
tokenizer_path = r"C:\Users\akash\Python_DL\PROJECT_DL\custom_tokenizer.json"
tokenizer = Tokenizer.from_file(tokenizer_path)

# If you want to further customize the tokenizer, you can set its attributes
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.decoder = decoders.WordPiece(prefix="##")
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0",
    special_tokens=[("[CLS]", 0), ("[SEP]", 1)],
)
tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace(r"[^\w\s]", ""),  # Replace non-alphanumeric characters with an empty string
        normalizers.Replace(r"\s+", " "),  # Replace multiple whitespaces with a single space
        normalizers.Lowercase(),
        normalizers.NFD(),
        normalizers.StripAccents(),
    ]
)

# Now you can use the tokenizer to tokenize and encode your examples
example_text = "This is a test example for the custom tokenizer."
encoded_example = tokenizer.encode(example_text)
print(encoded_example.tokens)


['[CLS]', 'this', 'is', 'a', 'test', 'exam', '##ple', 'for', 'the', 'custom', 'token', '##izer', '[UNK]', '[SEP]']
